<a href="https://colab.research.google.com/github/halen48/SocketsDemo/blob/main/Cliente_Servidor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dependências

><a href="https://docs.python.org/dev/library/socket.html#socket.socket.listen"> Documentação de Socket</a>

In [ ]:
import socket

>Para simular que os sistemas distribuidos, serão abertos dois processos na máquina local

In [ ]:
from multiprocessing import Process

>Cada conexão do lado do servidor é tratada com uma thread<br>
>Existe uma alternativa monothread, utilizando a função <a href="https://docs.python.org/3/library/select.html"><i>select</i></a>. <br>
>Cada sistema operacional oferece uma API específica para o tratamento de sockets, sendo assim: 
* <a href="https://docs.microsoft.com/en-us/windows/win32/api/winsock2/nf-winsock2-select">Windows</a>
* <a href="https://man7.org/linux/man-pages/man2/select.2.html">Linux</a>
* <a href="https://developer.apple.com/library/archive/documentation/System/Conceptual/ManPages_iPhoneOS/man2/select.2.html">MacOS</a>
* <a href="https://www.freebsd.org/cgi/man.cgi?select">FreeBSD</a>

In [ ]:
from threading import Thread, Lock

>Operações de socket são feitas em byte...<br>
>Python é um pouco problemático com isso, então usaremos a biblioteca <a href="https://docs.python.org/3/library/pickle.html">Pickle</a>

In [ ]:
import pickle

>Para coisas interessantes...

In [ ]:
import time
import random

>Vamos colocar uma velocidade de execução para as ações entre cliente-servidor.

In [ ]:
velocidade = 0.3 #@param {type:"slider", min:0.1, max:2, step:0.1}

#Servidor

##parse_cliente(self, conn, addr, id)

>Qual o motivo de você estar conectando um cliente com o servidor?<br>
>Ou seja, como o cliente vai se comportar no lado do servidor

In [ ]:
def parse_cliente(self, conn, addr, id):
  while True:
    try:

      data = conn.recv(tamanho_buffer)
      print('\033[3%dm'%(1+id%5)+'[Servidor] recebi', repr(data), 'do cliente <id: %d>'%id,'\x1b[0m')
      if not data:
          break
      
      data = data.decode(self.codificacao)

      valor = random.randint(0,50)

      if(data == 'negativo'):
        valor *= -1
      
      data = pickle.dumps( (id,valor) )
      time.sleep(0.5/velocidade)
      conn.sendall(data)
    except Exception as e:
      print("[Servidor]", e)
      continue

##gerenciador_clientes(self)

>Rotina principal do servidor
>Como os clientes vão se conectar ao servidor

In [ ]:
def gerenciador_clientes(self):
  with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    while True:
      try:
        s.bind(self.endereco)
        s.listen(self.max_clients)
        time.sleep(0.5/velocidade)
        break
      except Exception as e:
        print(e)
        print("Tentando vincular conexão...")
        continue
    
    while True:
      
      self.checar_conexoes_clientes(self)  

      conn, addr = s.accept()
      print('[Servidor] Foi conectado um cliente em:', addr)
      self.adicionar_cliente(self, conn, addr)
    #s.close()

##def adicionar_cliente(self, conn, addr)

>Rotina para adicionar um cliente no servidor

In [ ]:
def adicionar_cliente(self, conn, addr):

  id = self.ultimo_id
  self.ultimo_id += 1
  thread = Thread(target = self.parse_cliente, args=(self, conn, addr, id))
  self.lista_clientes[id] = ( (conn, thread) )
  self.lista_clientes[id][1].start()

  print("[Servidor] %d/%d clientes conectados"%(len(self.lista_clientes), self.max_clients))

  return id

##checar_conexoes_clientes(self)

>Checa todas as conexões conectadas ao servidor:<br>
>* Recusa novas conexões caso o servidor esteja cheio<br>
>* Quando um cliente é desconectado (thread acabou sua tarefa), esta rotina está encarregada em chamar a rotina para deletar o registro do servidor

In [ ]:
def checar_conexoes_clientes(self):
    while True:
      #limpa os clientes que estão desconectados
      self.clientes_conectados = list(self.lista_clientes.keys())
      for c_id in self.clientes_conectados:
        if (not self.lista_clientes[c_id][1].is_alive()):
          self.remover_cliente(self,c_id)
      #Caso a lista tenha espaço para mais clientes, a rotina encerra por aqui
      if (len(self.lista_clientes) < self.max_clients):
          return
      #Se tem mais clientes que o servidor pode suportar, aguarda...
      print("[Servidor] Muitos clientes conectados. Aguardando...")
      print("[Servidor - Debug] IDs Conectados: ", self.clientes_conectados)
      time.sleep(0.5/velocidade)

##def remover_cliente(self,index)

>Remove o registro do servidor, passando um ID como referência

In [ ]:
def remover_cliente(self,index):
  print('[Servidor] Desconectando o cliente[%d]: %s'%(index,self.lista_clientes[index][0]))
  self.lista_clientes[index][0].close()
  del self.lista_clientes[index]
  print("[Servidor] %d/%d clientes conectados"%(len(self.lista_clientes), self.max_clients))
  

##class Server()

In [ ]:
class Server():
  def __init__(self, end, cod, max_clients, tamanho_buffer):
    self.endereco = end
    self.codificacao = cod
    self.max_clients = max_clients
    self.lista_clientes = {}
    self.mutex = Lock()
    self.tamanho_buffer = tamanho_buffer
    self.ultimo_id = 0
    self.clientes_conectados = []

    #=========================================
    #=============== Funções =================
    #=========================================

    self.adicionar_cliente = adicionar_cliente
    self.checar_conexoes_clientes = checar_conexoes_clientes
    self.gerenciador_clientes = gerenciador_clientes
    self.remover_cliente = remover_cliente
    self.parse_cliente = parse_cliente
  
  def run(self):
    self.gerenciador_clientes(self)


#Cliente

##connect(self, s)

>Rotina para conectar o cliente ao servidor<br>
>É importante notar o loop "while True", pois a requisição de conexão pode ser recusada em algum momento.

In [ ]:
def connect(self, s):
  while True:
    try:
      s.connect(self.endereco)
      break
    except ConnectionRefusedError:
      #print('[Cliente] Tentando conectar...')
      time.sleep(2)
    except TimeoutError:
      if(self.id):
        print('\033[3%dm'%(1+self.id%5)+'[Cliente] Timeout! <id:%d>'%self.id,'\x1b[0m')
      else:
        print("[Cliente] Timeout! <id: Não conectado>")

##def rotina(self):

>Rotina principal do cliente<br>
>Aqui é onde o usuário pode efetuar suas ações para que o servidor processe.<br>
>Existem várias formas de tratar o encerramento de um socket, depende da aplicação e do protocolo abordado. 
>Neste caso, estamos encerrando o socket assim que o cliente não possui mais nada para fazer no lado do servidor, encerrando sua sessão.

In [ ]:
def rotina(self):
  with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    self.connect(s)
    while True:
      time.sleep(1/velocidade)
      mensagem = random.choice(['positivo','negativo'])
      s.sendall(bytes(mensagem,self.codificacao))
      data = pickle.loads ( s.recv(tamanho_buffer) )

      if( not self.id):
        self.id = data[0]
      
      print('\033[3%dm'%(1+self.id%5)+'[Cliente] recebi', data[1], 'do servidor <id:%d>'%self.id,'\x1b[0m')
      if(random.randint(0,5) == 0):
        print('\033[3%dm'%(1+self.id%5)+'[Cliente] vou desligar... <id:%d>'%self.id,'\x1b[0m')
        break
    #s.close()  

##class Client()

In [ ]:
class Client():
  def __init__(self, end, cod, tamanho_buffer):
    self.endereco = end
    self.codificacao = cod
    self.tamanho_buffer = tamanho_buffer
    self.id = None

    #=========================================
    #=============== Funções =================
    #=========================================

    self.connect = connect
    self.rotina = rotina

  def run(self):
    self.rotina(self)

#Rotina Principal

In [ ]:
endereco = ('127.0.0.1', 33333)
codificacao = 'latin-1'
tamanho_buffer = 2048

s = Server(endereco,codificacao, 5, tamanho_buffer)

p = Process(target=s.run)
p.start()

while True:
  time.sleep(random.randint(1,3)/5/velocidade)
  Thread(target = Client(endereco,codificacao, tamanho_buffer).run ).start()
  
  
p.join()